In [4]:
from selenium import webdriver
import lxml.html
import csv


#自动打开edge，获取代码
driver = webdriver.Edge('C:\Program Files (x86)\Microsoft\Edge Beta\Application\edgedriver_win32\msedgedriver.exe') 
##这里是下载的edge驱动地址，可以自己去官网下驱动
driver.get('http://2018.cctv.com/medal_list/details/index.shtml?countryid=NOR')  ##数据来源央视2018平昌东奥会挪威奖牌榜
content=driver.page_source  ##获取页面代码信息
driver.quit() ##关闭edge

f = open('nation_data_NOR.csv', 'w' , encoding='utf-8')  ##打开csv文件选择utf-8格式，后续可能会有乱码现象
csv_writer = csv.writer(f)

metree = lxml.html.etree  ##继续使用lxml，因为这里只用爬取一个页面，没必要用正则
parser = metree.HTML(content)
td_list_0 = parser.xpath("/html/body/div[3]/div/div/div/div/div[2]/div[3]/div/div/div/div[2]/div[1]/table/tbody//tr") ##使用xpath定位金牌
td_list_1 = parser.xpath("/html/body/div[3]/div/div/div/div/div[2]/div[3]/div/div/div/div[2]/div[2]/table/tbody//tr") ##使用xpath定位银牌
td_list_2 = parser.xpath("/html/body/div[3]/div/div/div/div/div[2]/div[3]/div/div/div/div[2]/div[3]/table/tbody//tr") ##使用xpath定位铜牌

##将爬取的金牌信息写入csv表中
for td_item in td_list_0:
    num_item= td_item.xpath('.//text()')
    csv_writer.writerow(num_item)

##将爬取的银牌信息写入csv表中
for td_item in range(1,len(td_list_1)):
    num_item= td_list_1[td_item].xpath('.//text()')
    csv_writer.writerow(num_item)

##将爬取的铜牌信息写入csv表中
for td_item in range(1,len(td_list_2)):
    num_item= td_list_2[td_item].xpath('.//text()')
    csv_writer.writerow(num_item)

f.close()  #关闭文件

##接下来对爬取的csv文件手动解决乱码并去除空行并转换为xlsx格式文件
##大概就是先用记事本打开将utf-8编码改为ansi，然后用excel表打开，使用ctrl+g去除多余的空行
##如果不好改，可以直接去github上下载数据集


In [6]:
from pyecharts import options as opts
from pyecharts.charts import Map
from pyecharts.charts import Pie
import pandas as pd
import os

datas=pd.read_excel('data_to_Piechar.xlsx')  ##读取excel表，这里是自己手动将上面爬取存储csv文件转换成xlsx文件
tmp = datas["大项"].value_counts()  ##统计每个大项分别得了多少奖牌（不区分金银铜）
key = list(tmp.keys())  ##key表示大项名称，用一个列表存
value = list(tmp)  ##value表示奖牌个数，用一个列表存


# 打开html
c = (
    Pie(init_opts = opts.InitOpts())
    .add("", [list(z) for z in zip(key, value)], radius=["10%", "120"],)  ##添加对应的大项和奖牌数信息，并调整饼状图的内外圈大小比列
    .set_global_opts(
        title_opts=opts.TitleOpts(title="挪威队"),  ##添加标题挪威队
        legend_opts=opts.LegendOpts(orient="vertical", pos_top="15%", pos_left="2%"),  ##调整位置
    )
    .set_series_opts(label_opts=opts.LabelOpts(formatter="{b} : {c}"))  ##默认格式即可
    .render(path="2018平昌冬奥会挪威队奖牌情况饼状图.html")
)

# 打开html
os.system("2018平昌冬奥会挪威队奖牌情况饼状图.html")


0